# **Sentiment Analysis on Tweets**
Sentiment analysis, also known as opinion mining, is the process of identifying and categorizing emotions expressed in text data—typically as positive, negative, or neutral. It helps organizations and individuals understand the sentiment behind user-generated content, such as product reviews, social media posts, or customer feedback.

In the context of social media, sentiment analysis is particularly valuable due to the vast amount of real-time user opinions shared daily. Twitter, with its concise and public messages, provides an ideal dataset for analyzing public sentiment around topics, events, brands, or products.

By leveraging natural language processing (NLP) techniques and machine learning models, sentiment analysis can extract insights from tweets to support business decisions, brand monitoring, political analysis, and crisis management.

This project focuses on building a sentiment classifier using a dataset of tweets. The model aims to classify each tweet as positive or negative, helping reveal how people feel about certain topics at scale.

<br>

**Dataset:** [Kaggle Sentiment140](https://www.kaggle.com/datasets/kazanova/sentiment140/data)

---

## **Data Preprocessing**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import emoji
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

Prepare the dataset.

In [ ]:
dataset_path = '../data/twt.csv'
column_names = ['sentiment', 'id', 'date', 'flag', 'user', 'text']
df = pd.read_csv(dataset_path, encoding='latin', delimiter=',', names=column_names)
df.head()

Drop unimportant columns and missing values.

In [ ]:
df = df.drop(['id', 'date', 'flag', 'user'], axis=1)
df = df.dropna()
df.head()

### Preprocess the data:
- convert the emojis into text
- lowercase everything
- remove urls, mentions and hashtags
- remove punctuations and special characters
- remove stopwords
- split into tokens

In [ ]:
print("Sample stopwords being removed:")
print(list(stop_words))

In [ ]:
sentiment_critical = {
    'not', 'no', 'never', 'nothing', 'nobody', 'none', 'nowhere', 'neither',
    'very', 'really', 'quite', 'rather', 'extremely', 'incredibly', 'absolutely',
    'but', 'however', 'although', 'though', 'yet', 'except',
    'too', 'so', 'such', 'more', 'most', 'less', 'least',
    'only', 'just', 'still', 'even', 'again'
}

negative_contractions = {
    "don't", "won't", "can't", "shouldn't", "wouldn't", "couldn't",
    "isn't", "aren't", "wasn't", "weren't", "hasn't", "haven't",
    "hadn't", "doesn't", "didn't", "won't", "shan't", "mustn't",
    "mightn't", "needn't"
}

sentiment_critical.update(negative_contractions)

In [ ]:
def clean_twts(twt):
    twt = twt.lower()
    twt = re.sub(r"http\S+|www\S+|https\S+", '', twt)  # remove urls
    twt = re.sub(r"@\w+", '', twt)  # remove mentions
    twt = re.sub(r"#", '', twt)  # remove hashtag symbol
    twt = emoji.replace_emoji(twt, replace='')  # remove emojis
    twt = re.sub(r"[^a-zA-Z\s']", '', twt)  # remove punctuation
    twt = re.sub(r"\s+", ' ', twt).strip()  # clean whitespace

    tokens = twt.split()
    tokens = [word for word in tokens if (word not in stop_words or word in sentiment_critical) and len(word) > 1] # remove stopwords and keep sentiment-critical words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # lemmatize

    return ' '.join(tokens)

cleaned_twts = df['text'].apply(clean_twts)
df['cleaned_text'] = cleaned_twts

In [ ]:
df.head()

In [ ]:
# Check average tweet length after preprocessing
lengths = [len(text.split()) for text in cleaned_twts]
print(f"Average length: {np.mean(lengths):.2f}")
print(f"95th percentile: {np.percentile(lengths, 95):.2f}")

Tokenize the data.

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(cleaned_twts)
sequences = tokenizer.texts_to_sequences(cleaned_twts)
padded_sequences = pad_sequences(sequences, maxlen=20, padding='post', truncating='post')

print("Tokenized and padded sequences:")
print(padded_sequences)

Add into the dataframe.

In [ ]:
df['padded_text'] = list(padded_sequences)
df['sentiment'] = df['sentiment'].map({4: 1, 0: 0})
df.head()

Split the dataset.

In [ ]:
x = padded_sequences
y = df['sentiment']


x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.4, random_state=42)

x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=10000, output_dim=64),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.build(input_shape=(None,20))

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=12,
    batch_size=32,
    validation_data=(x_val, y_val)
)

In [ ]:
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.title("Model Accuracy")
plt.show()

In [ ]:
preds = model.predict(x_test)
preds_binary = (preds > 0.5).astype(int).flatten()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(y_test, preds_binary))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, preds_binary))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print(f"\nAccuracy: {accuracy_score(y_test, preds_binary):.4f}")
print(f"Precision: {precision_score(y_test, preds_binary):.4f}")
print(f"Recall: {recall_score(y_test, preds_binary):.4f}")
print(f"F1-Score: {f1_score(y_test, preds_binary):.4f}")

In [ ]:
cm = confusion_matrix(y_test, preds_binary)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

---

**Initial training results:** the model can be improved. Will try again.  
**Second attempt:** model improved slightly. Will try again.  
**Third attempt:** tried adding more epochs and batches. Will try again.  
**Fourth attempt:** used lemmatization, better results. Will try again.
**Fifth attempt:** improved stopword removal, model now performing decently. Will try again.  
**Sixth attempt:** hyperparameter tuning using manual randomsearch and cross validation with stratified kfold. Will try to tune again.

---